# Brain tumor detection using tensorflow and opencv

In [70]:
# importing libraries

import os
import tensorflow as tf
from keras import layers
from keras import Model
from keras.applications import InceptionV3
import numpy as np
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [71]:
# saving the location of the base directory

base_dir = r"C:\Users\91976\Desktop\programming\AI and Ml\projects\survival predicton\Tumor"

In [72]:
# storing the location of the training  and testing directories 

test_dir = f'{base_dir}\images\Testing'
train_dir = f"{base_dir}\images\Training"

train_glioma_tumor_dir = f'{train_dir}\glioma_tumor'
train_meningioma_tumor_dir = f'{train_dir}\meningioma_tumor'
train_no_tumor_dir = fr'{train_dir}\no_tumor'
train_pituitary_tumor_dir = f'{train_dir}\pituitary_tumor'


test_glioma_tumor_dir = f'{test_dir}\glioma_tumor'
test_meningioma_tumor_dir = f'{test_dir}\meningioma_tumor'
test_no_tumor_dir = fr"{test_dir}\no_tumor"
test_pituitary_tumor_dir = f'{test_dir}\pituitary_tumor'



In [73]:
# using opencv to load images and resizing grayscale images to rgb and labeling the images


from keras.preprocessing.image import ImageDataGenerator

labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']


X_train = []
y_train = []



image_size = 224
for i in labels:
    folderPath = f"{train_dir}\{i}"
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
       
for i in labels:
    folderPath = f"{test_dir}\{i}"
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)


        
X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape)

100%|██████████| 74/74 [00:00<00:00, 158.31it/s]


(3264, 224, 224, 3)


In [74]:
# spittting the data into test and train for validation

X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=101)


y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)



print(X_test.shape)
print(X_train.shape)
print(y_train.shape)
print(y_test.shape)



(327, 224, 224, 3)
(2937, 224, 224, 3)
(2937, 4)
(327, 4)


# importing pre trained model InceptionV3

In [75]:


pre_model = InceptionV3(
        input_shape=(224,224,3),
        weights='imagenet',   
        include_top=False,
            
)

'''
pre_model = VGG16(
        input_shape=(224,224,3),
        weights='imagenet',
        include_top=False,
        pooling='max'
)
'''

# freezing the layers 

for layers in pre_model.layers:
    layers.trainable=False


# using the mixed7 layer for output of inceptionv3

last_layer = pre_model.get_layer('mixed7')
last_output = last_layer.output
  

# Model formation and training

In [76]:
from keras.optimizers import Adam,RMSprop


x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(128,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
output = tf.keras.layers.Dense(4,activation='softmax')(x)

model = Model(pre_model.input,output)

model.compile(optimizer=Adam(learning_rate=0.001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy']
)

In [77]:
# stopping the training as soon as the val_accuracy reaches 0.95


class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('val_accuracy')>0.95):
            self.model.stop_training = True

training the model

In [78]:



callback = myCallbacks()

datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

datagen_val = ImageDataGenerator(rescale=1./255)

 

model.fit(

    datagen.flow(X_train,y_train, batch_size=32),
    epochs=10,
    validation_data=datagen_val.flow(X_test,y_test),
    steps_per_epoch = len(X_train) / 32 ,
    callbacks=[callback]
)

Epoch 1/10
91/91 [==============================] - 106s 1s/step - loss: 0.7146 - accuracy: 0.7303 - val_loss: 0.6676 - val_accuracy: 0.6972
Epoch 2/10
91/91 [==============================] - 89s 968ms/step - loss: 0.3581 - accuracy: 0.8662 - val_loss: 0.2976 - val_accuracy: 0.8869
Epoch 3/10
91/91 [==============================] - 89s 967ms/step - loss: 0.2651 - accuracy: 0.9043 - val_loss: 0.2663 - val_accuracy: 0.8930
Epoch 4/10
91/91 [==============================] - 88s 960ms/step - loss: 0.1875 - accuracy: 0.9309 - val_loss: 0.3819 - val_accuracy: 0.8869
Epoch 5/10
91/91 [==============================] - 85s 918ms/step - loss: 0.1721 - accuracy: 0.9414 - val_loss: 0.1851 - val_accuracy: 0.9297
Epoch 6/10
91/91 [==============================] - 106s 1s/step - loss: 0.1374 - accuracy: 0.9544 - val_loss: 0.2178 - val_accuracy: 0.9266
Epoch 7/10
91/91 [==============================] - 96s 1s/step - loss: 0.1182 - accuracy: 0.9591 - val_loss: 0.2777 - val_accuracy: 0.8899
Epoch 

# Saving the model

In [79]:
#model.save('tumor_cv.h5')

In [80]:

'''
tf_model_file = "tumor_cv.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

with open(tf_model_file,'wb') as file:
    file.write(tflite_model)
'''

'\ntf_model_file = "tumor_cv.tflite"\ntf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)\ntflite_model = tf_lite_converter.convert()\n\nwith open(tf_model_file,\'wb\') as file:\n    file.write(tflite_model)\n'

Testing

In [81]:
t = []

img = cv2.imread(r"C:\Users\91976\Desktop\programming\AI and Ml\projects\survival predicton\Tumor\images\Testing\pituitary_tumor\image(1).jpg")

img = cv2.resize(img,(224,224))
t.append(img)

t_p = np.array(t)

In [82]:
t_p = t_p / 255.0
pred = model.predict(t_p)

1/1 [==============================] - 2s 2s/step


In [83]:
print(pred)
predictions = np.argmax(pred,1)


[[1.8080780e-03 1.8929919e-05 8.4695005e-04 9.9732602e-01]]


In [84]:
'''
labels = {

    'glioma_tumor': 0,
    'no_tumor' : 1,
    'meningioma_tumor': 2,
    'pituitary_tumor' : 3


}


'''

"\nlabels = {\n\n    'glioma_tumor': 0,\n    'no_tumor' : 1,\n    'meningioma_tumor': 2,\n    'pituitary_tumor' : 3\n\n\n}\n\n\n"